# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../WeatherPy/cities.csv")

# Display sample data
city_data_df.head()

,ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,margaret river,-33.9500,115.0667,19.84,52,87,4.96,AU,1701022380
1,1,crane,31.3974,-102.3501,8.44,35,100,5.60,US,1701022381
2,2,walvis bay,-22.9575,14.5053,16.16,70,26,6.76,NaN,1701022377
3,3,gamay,12.3854,125.2979,25.23,93,33,2.12,PH,1701022383
4,4,igarka,67.4667,86.5833,-27.50,99,80,2.58,RU,1701022384


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
city_data_map = city_data_df.hvplot.points("Lng", "Lat", geo = True, tiles = "OSM", frame_width = 800, frame_height = 600, 
                                           size = "Humidity", alpha = 0.5, color = 'Country', xlabel = 'Longitude', ylabel = 'Latitude')

# Display the map
city_data_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Country,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
hightemp = city_data_df.loc[city_data_df['Max Temp']<29,:]
lowtemp = hightemp.loc[hightemp['Max Temp']>22,:]
windspeed = lowtemp.loc[lowtemp['Wind Speed']<6,:]
idealweather = windspeed.loc[windspeed['Cloudiness']<20,:]

# Drop any rows with null values
idealweather_drop = idealweather.dropna()

# Display sample data
idealweather_drop

,ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
29,29,calvinia,-31.4707,19.7760,25.87,23,6,3.24,ZA,1701022404
38,38,amboanjo,-22.6667,47.8167,24.72,79,3,4.88,MG,1701022411
61,61,al kharj,24.1550,47.3346,25.32,43,9,4.66,SA,1701022430
68,68,hawaiian paradise park,19.5933,-154.9731,23.28,94,0,2.06,US,1701022435
134,134,tolanaro,-25.0319,46.9987,24.52,87,5,5.93,MG,1701022490
138,138,bubaque,11.2833,-15.8333,27.37,62,1,3.27,GW,1701022493
202,202,hilo,19.7297,-155.0900,24.23,87,0,2.06,US,1701022546
312,312,ar rass,25.8694,43.4973,24.11,41,12,0.54,SA,1701022641
379,379,alofi,-19.0595,-169.9187,25.94,94,14,2.06,NU,1701022697
389,389,pisco,-13.7000,-76.2167,23.03,64,0,3.60,PE,1701022706


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = idealweather_drop.loc[:,['City','Country','Lat','Lng','Humidity']]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
29,calvinia,ZA,-31.4707,19.7760,23,
38,amboanjo,MG,-22.6667,47.8167,79,
61,al kharj,SA,24.1550,47.3346,43,
68,hawaiian paradise park,US,19.5933,-154.9731,94,
134,tolanaro,MG,-25.0319,46.9987,87,
138,bubaque,GW,11.2833,-15.8333,62,
202,hilo,US,19.7297,-155.0900,87,
312,ar rass,SA,25.8694,43.4973,41,
379,alofi,NU,-19.0595,-169.9187,94,
389,pisco,PE,-13.7000,-76.2167,64,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey":geoapify_key,
    "categories":"accommodation.hotel"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, ID in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = ID['Lat']
    lon = ID['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
calvinia - nearest hotel: No hotel found
amboanjo - nearest hotel: No hotel found
al kharj - nearest hotel: شقق فندقية
hawaiian paradise park - nearest hotel: No hotel found
tolanaro - nearest hotel: Hôtel Mahavokey
bubaque - nearest hotel: Chez Julio
hilo - nearest hotel: Dolphin Bay Hotel
ar rass - nearest hotel: Saif Boutique Hotel International
alofi - nearest hotel: Matavai Resort
pisco - nearest hotel: La Portada
salto - nearest hotel: Hotel Español Salto
farafangana - nearest hotel: Le Concombre Masqué
ambovombe - nearest hotel: La source
al hufuf - nearest hotel: Al Muhaidab Residence Al Ahsa
alibag - nearest hotel: Chirag executive
toliara - nearest hotel: Ambary
al ghayzah - nearest hotel: فندق تاج العرب
haiku-pauwela - nearest hotel: Inn At Mama's Fish House
chivhu - nearest hotel: Vic's Travern
chibuto - nearest hotel: DESHENG


,City,Country,Lat,Lng,Humidity,Hotel Name
29,calvinia,ZA,-31.4707,19.7760,23,No hotel found
38,amboanjo,MG,-22.6667,47.8167,79,No hotel found
61,al kharj,SA,24.1550,47.3346,43,شقق فندقية
68,hawaiian paradise park,US,19.5933,-154.9731,94,No hotel found
134,tolanaro,MG,-25.0319,46.9987,87,Hôtel Mahavokey
138,bubaque,GW,11.2833,-15.8333,62,Chez Julio
202,hilo,US,19.7297,-155.0900,87,Dolphin Bay Hotel
312,ar rass,SA,25.8694,43.4973,41,Saif Boutique Hotel International
379,alofi,NU,-19.0595,-169.9187,94,Matavai Resort
389,pisco,PE,-13.7000,-76.2167,64,La Portada


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points("Lng", "Lat", geo = True,tiles = "OSM", frame_width = 800, frame_height = 600, size = "Humidity",
                                   alpha =0.5, color = 'Country', hover_cols = ["Hotel Name", "Country"], xlabel = 'Longitude', 
                                   ylabel = 'Latitude')

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Country,Humidity,Hotel Name)